Описание задачи:  
    Есть файл csv с оценками документов асессорами.  
    Формат строки данных: worker_id, document_id, label  
    worker_id, document_id - целые числа  
    label - строка  
    Гарантируется, что асессор не может оценить один и тот же документ 2 раза.  
    
Необходимо посчитать точность для каждого асессора в формате:  
    worker_id, accuracy,  
    где accuracy - это процент правильных меток, поставленных данным асессором.  
    Под правильной меткой понимается та, которую поставило большинство асессоров для данного документа.  
    Если кол-во разных меток по одному документу одинаковое, то какая-то из них случайно назначается правильной,  
    а остальные - неправильными

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('dataset.csv')

In [3]:
df

,worker_id,document_id,label
0,11,777,A
1,22,777,B
2,33,777,A
3,11,888,good
4,22,888,good
5,33,888,good
6,44,888,bad
7,11,999,good
8,22,999,good
9,33,999,bad


In [4]:
# Подсчет количества меток к каждому документу
count_df = df.groupby(['document_id', 'label']).count().reset_index() \
    .rename({'worker_id': 'labels_count'}, axis=1)

In [5]:
count_df

,document_id,label,labels_count
0,111,A,1
1,111,B,1
2,777,A,2
3,777,B,1
4,888,bad,1
5,888,good,3
6,999,bad,1
7,999,good,3
8,999,unknown,1


In [6]:
# Определение правильной метки (той, которую поставили максимальное число раз)
count_df = count_df.iloc[count_df.groupby(['document_id']).apply(lambda x: x['labels_count'].idxmax())]

In [7]:
count_df

,document_id,label,labels_count
0,111,A,1
2,777,A,2
5,888,good,3
7,999,good,3


In [8]:
# Объединяю основной фрейм с правильными оценками.
# На месте неправильных будет NaN
df = pd.merge(df, count_df, on=['document_id', 'label'], how='left')

In [9]:
df

,worker_id,document_id,label,labels_count
0,11,777,A,2.0
1,22,777,B,NaN
2,33,777,A,2.0
3,11,888,good,3.0
4,22,888,good,3.0
5,33,888,good,3.0
6,44,888,bad,NaN
7,11,999,good,3.0
8,22,999,good,3.0
9,33,999,bad,NaN


In [10]:
# Фрейм только с правильными оценками
df_correct = df.dropna(subset=['labels_count'])

In [11]:
df_correct

,worker_id,document_id,label,labels_count
0,11,777,A,2.0
2,33,777,A,2.0
3,11,888,good,3.0
4,22,888,good,3.0
5,33,888,good,3.0
7,11,999,good,3.0
8,22,999,good,3.0
10,44,999,good,3.0
12,44,111,A,1.0


In [12]:
# Кол-во документов, оцененных пользователем всего
res_df = df.groupby('worker_id')['label'].count().reset_index() \
    .rename({'label': 'total_docs_count'}, axis=1)

In [13]:
res_df

,worker_id,total_docs_count
0,11,3
1,22,3
2,33,3
3,44,3
4,55,2


In [14]:
# Кол-во документов, оцененных пользователем правильно
res_df2 = df_correct.groupby('worker_id')['label'].count().reset_index() \
    .rename({'label': 'correct_docs_count'}, axis=1)

In [15]:
res_df2

,worker_id,correct_docs_count
0,11,3
1,22,2
2,33,2
3,44,2


In [16]:
# Итоговый фрейм с результатами работы каждого асессора (в столбце accuracy)
res_df = pd.merge(res_df, res_df2, on='worker_id', how='left')
res_df['correct_docs_count'] =res_df['correct_docs_count'].fillna(value=0)
res_df['accuracy'] = round(100 * res_df['correct_docs_count'] / res_df['total_docs_count'], 2)

In [17]:
res_df

,worker_id,total_docs_count,correct_docs_count,accuracy
0,11,3,3.0,100.00
1,22,3,2.0,66.67
2,33,3,2.0,66.67
3,44,3,2.0,66.67
4,55,2,0.0,0.00
